In [4]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
import prepare as p


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kaylabrock/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/kaylabrock/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kaylabrock/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
df = pd.read_json('data.json')

In [6]:
df = p.create_df(df)

In [7]:
df.head()

,repo,language,original,lemmatized
0,acidanthera/AppleALC,C++,AppleALC\n========\n\n[![Build Status](https:/...,applealc acidanthera applealc workflow ci badg...
1,gongjianhui/AppleDNS,Python,# Final AppleDNS Pro\n\nAppleDNS 通过收集 Apple 在中...,final appledns pro appledns cdn itunes icloud ...
2,tomaz/appledoc,Objective-C,About appledoc\n==============\n\n**IMPORTANT ...,appledoc important notice collaborator needed ...
3,robovm/apple-ios-samples,Objective-C,# Mirror of Apple's iOS samples\n\nThis reposi...,mirror io sample repository mirror io sample l...
4,appleseedhq/appleseed,C++,# appleseed [![Build Status](https://travis-ci...,appleseed travis ci appleseedhq appleseed svg ...


In [8]:
df.shape

(368, 4)

In [9]:
df.language.value_counts()

Swift          76
JavaScript     59
Objective-C    56
Python         45
C              34
Java           30
C++            20
Shell          19
PHP            18
C#             11
Name: language, dtype: int64

In [10]:
df.dtypes

repo          object
language      object
original      object
lemmatized    object
dtype: object

In [11]:
df

,repo,language,original,lemmatized
0,acidanthera/AppleALC,C++,AppleALC\n========\n\n[![Build Status](https:/...,applealc acidanthera applealc workflow ci badg...
1,gongjianhui/AppleDNS,Python,# Final AppleDNS Pro\n\nAppleDNS 通过收集 Apple 在中...,final appledns pro appledns cdn itunes icloud ...
2,tomaz/appledoc,Objective-C,About appledoc\n==============\n\n**IMPORTANT ...,appledoc important notice collaborator needed ...
3,robovm/apple-ios-samples,Objective-C,# Mirror of Apple's iOS samples\n\nThis reposi...,mirror io sample repository mirror io sample l...
4,appleseedhq/appleseed,C++,# appleseed [![Build Status](https://travis-ci...,appleseed travis ci appleseedhq appleseed svg ...
...,...,...,...,...
481,PatMurrayDEV/apple-music-history,JavaScript,[![Netlify Status](https://api.netlify.com/api...,netlify api netlify api v1 badge e207d990 1ee1...
482,antongorodezkiy/codeigniter-apns,PHP,"Codeigniter-apns\n(c) 2012, Anton Gorodezkiy\n...",codeigniter apns c 2012 anton gorodezkiy codei...
484,lprhodes/homebridge-apple-tv,JavaScript,# Homebridge Apple TV\n\n## Introduction\nWelc...,homebridge tv welcome tv plugin homebridge nfa...
485,lvsti/CoreMediaIO-DAL-Example,C++,# CoreMediaIO-DAL-Example\n\nModernized and ex...,coremediaio dal example modernized extended co...


In [12]:
df['lemmatized'] = df['lemmatized'].astype('str') 

In [28]:
all_lemmatized = ' '.join(df.lemmatized)
Swift_lemmatized = ' '.join(df[df.language == 'Swift'].lemmatized)
Javascript_lemmatized = ' '.join(df[df.language == 'JavaScript'].lemmatized)
objectiveC_lemmatized = ' '.join(df[df.language == 'Objective-C'].lemmatized)
Python_lemmatized = ' '.join(df[df.language == 'Python'].lemmatized)
C_lemmatized = ' '.join(df[df.language == 'C'].lemmatized)
Java_lemmatized = ' '.join(df[df.language == 'Java'].lemmatized)
Cplusplus_lemmatized = ' '.join(df[df.language == 'C++'].lemmatized)
Shell_lemmatized = ' '.join(df[df.language == 'Shell'].lemmatized)
PHP_lemmatized = ' '.join(df[df.language == 'PHP'].lemmatized)
Cpound_lemmatized = ' '.join(df[df.language == 'C#'].lemmatized)

In [29]:
All_freq = pd.Series(all_lemmatized.split()).value_counts()
Swift_freq = pd.Series(Swift_lemmatized.split()).value_counts()
Javascript_freq = pd.Series(Javascript_lemmatized.split()).value_counts()
ObjectiveC_freq = pd.Series(objectiveC_lemmatized.split()).value_counts()
Python_freq = pd.Series(Python_lemmatized.split()).value_counts()
C_freq = pd.Series(C_lemmatized.split()).value_counts()
Java_freq = pd.Series(Java_lemmatized.split()).value_counts()
Cplusplus_freq = pd.Series(Cplusplus_lemmatized.split()).value_counts()
Shell_freq = pd.Series(Shell_lemmatized.split()).value_counts()
PHP_freq = pd.Series(PHP_lemmatized.split()).value_counts()
Cpound_freq = pd.Series(Cpound_lemmatized.split()).value_counts()

In [30]:
word_counts = pd.concat([All_freq, Swift_freq, Javascript_freq, ObjectiveC_freq, Python_freq, C_freq, Java_freq, Cplusplus_freq, Shell_freq, PHP_freq, Cpound_freq], sort=True, axis=1)
word_counts.columns = ['All', 'Swift', 'Javascript', 'ObjectiveC', 'Python', 'C', 'Java', 'C++', 'Shell', 'PHP', 'C#']
word_counts = word_counts.fillna(0).apply(lambda s: s.astype(int))

In [34]:
word_counts.sort_values(by= 'All', ascending=False).head()

,All,Swift,Javascript,ObjectiveC,Python,C,Java,C++,Shell,PHP,C#
io,2368,623,555,535,106,37,68,16,363,28,37
app,1842,407,833,328,49,22,54,4,83,26,36
1,1489,288,394,171,167,143,84,51,144,18,29
2,1325,194,467,135,86,150,87,34,137,17,18
img,1163,367,92,82,33,19,38,22,430,16,64


In [35]:
word_counts.sort_values(by= 'Swift', ascending=False).head()

,All,Swift,Javascript,ObjectiveC,Python,C,Java,C++,Shell,PHP,C#
io,2368,623,555,535,106,37,68,16,363,28,37
swift,729,589,4,34,0,0,39,2,61,0,0
swiftui,485,470,1,0,0,0,0,0,14,0,0
app,1842,407,833,328,49,22,54,4,83,26,36
img,1163,367,92,82,33,19,38,22,430,16,64


In [36]:
word_counts.sort_values(by= 'Javascript', ascending=False).head()

,All,Swift,Javascript,ObjectiveC,Python,C,Java,C++,Shell,PHP,C#
freeware,869,0,868,0,0,0,0,0,1,0,0
app,1842,407,833,328,49,22,54,4,83,26,36
icon,982,19,831,17,16,6,3,0,85,0,5
yes,834,4,791,34,2,1,0,0,0,1,1
native,931,37,716,79,1,18,42,5,22,2,9


In [ ]:
('io app', 'sdds, sdd', '' == objective c 
('library', 'br', )

In [46]:
word_counts.sort_values(by= 'ObjectiveC', ascending=False).head(25)

,All,Swift,Javascript,ObjectiveC,Python,C,Java,C++,Shell,PHP,C#
io,2368,623,555,535,106,37,68,16,363,28,37
app,1842,407,833,328,49,22,54,4,83,26,36
library,798,106,69,321,58,49,30,51,76,22,16
use,1078,210,158,286,77,86,73,33,77,40,38
html,699,42,230,257,29,38,35,7,50,6,5
using,806,187,108,234,40,57,35,27,85,20,13
application,534,58,105,186,23,19,17,5,102,5,14
sample,290,26,6,181,18,24,4,7,6,10,8
br,532,50,73,173,15,10,40,13,84,1,73
1,1489,288,394,171,167,143,84,51,144,18,29


In [39]:
word_counts.sort_values(by= 'Python', ascending=False).head()

,All,Swift,Javascript,ObjectiveC,Python,C,Java,C++,Shell,PHP,C#
u,609,17,196,20,296,9,2,10,48,9,2
iphone,433,31,24,45,291,14,0,1,13,11,3
11,449,51,77,18,231,37,5,6,23,0,1
py,228,7,14,0,189,9,0,8,1,0,0
1,1489,288,394,171,167,143,84,51,144,18,29


In [40]:
word_counts.sort_values(by= 'C', ascending=False).head()

,All,Swift,Javascript,ObjectiveC,Python,C,Java,C++,Shell,PHP,C#
2,1325,194,467,135,86,150,87,34,137,17,18
1,1489,288,394,171,167,143,84,51,144,18,29
m1,324,3,113,1,2,103,0,2,100,0,0
homekit,135,0,4,5,0,95,0,30,1,0,0
file,939,155,154,113,129,94,72,50,99,40,33


In [41]:
word_counts.sort_values(by= 'Java', ascending=False).head()

,All,Swift,Javascript,ObjectiveC,Python,C,Java,C++,Shell,PHP,C#
java,119,0,19,1,1,1,92,0,5,0,0
tailor,90,0,0,0,0,0,90,0,0,0,0
2,1325,194,467,135,86,150,87,34,137,17,18
1,1489,288,394,171,167,143,84,51,144,18,29
use,1078,210,158,286,77,86,73,33,77,40,38


In [42]:
word_counts.sort_values(by= 'C++', ascending=False).head()

,All,Swift,Javascript,ObjectiveC,Python,C,Java,C++,Shell,PHP,C#
shelly,102,0,0,0,0,0,0,102,0,0,0
window,330,20,31,24,26,32,16,55,87,2,37
library,798,106,69,321,58,49,30,51,76,22,16
1,1489,288,394,171,167,143,84,51,144,18,29
file,939,155,154,113,129,94,72,50,99,40,33


In [43]:
word_counts.sort_values(by= 'Shell', ascending=False).head()

,All,Swift,Javascript,ObjectiveC,Python,C,Java,C++,Shell,PHP,C#
img,1163,367,92,82,33,19,38,22,430,16,64
io,2368,623,555,535,106,37,68,16,363,28,37
c,652,92,107,58,18,70,8,47,223,8,21
svg,582,197,56,35,22,8,21,14,202,20,7
badge,456,139,24,16,18,6,16,19,200,15,3


In [44]:
word_counts.sort_values(by= 'PHP', ascending=False).head()

,All,Swift,Javascript,ObjectiveC,Python,C,Java,C++,Shell,PHP,C#
php,169,5,50,2,2,20,6,0,1,83,0
news,162,24,47,3,0,0,1,0,4,80,3
post,187,13,86,15,8,5,3,0,3,51,3
key,426,18,85,58,57,72,51,3,10,41,31
file,939,155,154,113,129,94,72,50,99,40,33


In [45]:
word_counts.sort_values(by= 'C#', ascending=False).head()

,All,Swift,Javascript,ObjectiveC,Python,C,Java,C++,Shell,PHP,C#
br,532,50,73,173,15,10,40,13,84,1,73
img,1163,367,92,82,33,19,38,22,430,16,64
unity,63,0,2,0,1,0,0,1,9,0,50
aerial,71,22,0,0,0,0,0,0,0,0,49
heif,51,0,0,0,0,0,0,0,4,0,47


# Bigrams 

In [32]:
df['bigram'] = [list(nltk.bigrams(lemmatized.split())) for lemmatized in df.lemmatized]
df.head()

,repo,language,original,lemmatized,bigram
0,acidanthera/AppleALC,C++,AppleALC\n========\n\n[![Build Status](https:/...,applealc acidanthera applealc workflow ci badg...,"[(applealc, acidanthera), (acidanthera, applea..."
1,gongjianhui/AppleDNS,Python,# Final AppleDNS Pro\n\nAppleDNS 通过收集 Apple 在中...,final appledns pro appledns cdn itunes icloud ...,"[(final, appledns), (appledns, pro), (pro, app..."
2,tomaz/appledoc,Objective-C,About appledoc\n==============\n\n**IMPORTANT ...,appledoc important notice collaborator needed ...,"[(appledoc, important), (important, notice), (..."
3,robovm/apple-ios-samples,Objective-C,# Mirror of Apple's iOS samples\n\nThis reposi...,mirror io sample repository mirror io sample l...,"[(mirror, io), (io, sample), (sample, reposito..."
4,appleseedhq/appleseed,C++,# appleseed [![Build Status](https://travis-ci...,appleseed travis ci appleseedhq appleseed svg ...,"[(appleseed, travis), (travis, ci), (ci, apple..."


In [33]:
from wordcloud import WordCloud 
lemmatized_one = df.lemmatized[0]
lemmatized_one

ModuleNotFoundError: No module named 'wordcloud'